## Drop Athena DB and delete S3 bucket

In [14]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

/home/sagemaker-user/.conda/envs/data_science_on_aws/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### List existing S3 buckets

In [15]:
!aws s3 ls

2024-02-18 20:23:31 211125778552personalizepocvod
2024-02-18 01:27:18 aws-athena-query-results-211125778552-us-east-1
2024-04-13 17:52:38 aws-athena-query-results-us-east-1-211125778552
2024-03-27 00:35:44 aws-glue-assets-211125778552-us-east-1
2024-03-27 00:28:12 aws-glue-assets-211125778552-us-east-2
2024-02-15 21:19:44 sagemaker-studio-12jvao34qlkn
2024-02-15 22:38:05 sagemaker-studio-211125778552-3pjkfc2ijfr
2024-02-17 02:02:09 sagemaker-studio-211125778552-4dcj21sopi3
2024-02-19 03:02:29 sagemaker-studio-211125778552-4rfwbx1bibn
2024-02-15 20:23:46 sagemaker-studio-211125778552-4yhhjbuzjdq
2024-02-17 02:02:35 sagemaker-studio-211125778552-8xxlet4bnrv
2024-02-17 02:02:08 sagemaker-studio-211125778552-rfcwvtinree
2024-02-20 00:38:45 sagemaker-studio-211125778552-yu1t8p5304s
2024-03-21 19:20:47 sagemaker-studio-uyd2sz3oy3
2024-03-08 03:11:27 sagemaker-team11-stanford-dogs
2024-04-13 20:37:15 sagemaker-team4-bucket
2024-03-18 01:49:41 sagemaker-team6-distracted-drivers
2024-02-15 20:2

### Drop the Athena database

In [16]:
from pyathena import connect
import pandas as pd

In [17]:
database_name = "team4"
bucket = "sagemaker-team4-bucket"

In [18]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [19]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [20]:
statement = "DROP DATABASE IF EXISTS {} CASCADE".format(database_name)
print(statement)

DROP DATABASE IF EXISTS team4 CASCADE


In [21]:
pd.read_sql(statement, conn)

""


In [22]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,sagemaker_featurestore
3,team-8-fec-db
4,team3-court-data


### Remove the Ryanair customer feedback file from the S3 bucket

In [23]:
!aws s3 rm s3://sagemaker-team4-bucket/ryanair-data/ryanair_reviews.csv

delete: s3://sagemaker-team4-bucket/ryanair-data/ryanair_reviews.csv


### Remove the S3 bucket

In [24]:
!aws s3 rb s3://sagemaker-team4-bucket --force

delete: s3://sagemaker-team4-bucket/athena/staging/07fc18f2-8f89-4f8e-b7dd-f576c7a69593.csv.metadata
delete: s3://sagemaker-team4-bucket/athena/staging/130d9037-144e-4202-b297-ead61720622a.txt
delete: s3://sagemaker-team4-bucket/athena/staging/07fc18f2-8f89-4f8e-b7dd-f576c7a69593.csv
delete: s3://sagemaker-team4-bucket/athena/staging/24aa83a4-5a46-4b2b-9880-b3fa101b8e49.txt
delete: s3://sagemaker-team4-bucket/athena/staging/21a1cd5c-d895-4c12-a9a3-3bf20811be10.txt
delete: s3://sagemaker-team4-bucket/athena/staging/3094c9d8-2e97-4777-a260-76b12b25bede.csv.metadata
delete: s3://sagemaker-team4-bucket/athena/staging/2614fb8a-edeb-4e3f-b5db-3cb6a78466f0.txt
delete: s3://sagemaker-team4-bucket/athena/staging/2eb62712-76bd-4552-a1e9-759ee2b91333.txt.metadata
delete: s3://sagemaker-team4-bucket/athena/staging/4893d8b1-dc7d-47ce-915d-91942e5edd18.txt
delete: s3://sagemaker-team4-bucket/athena/staging/5d454fab-14ef-43cd-88ea-0d355010083a.txt
delete: s3://sagemaker-team4-bucket/athena/staging/2d